# Learning a Neural Network

Now, let's actually try to learn a ML model: as a first example, we will consider neural networks and train them using [keras](https://keras.io) and [TensorFlow](https://www.tensorflow.org)

Given our input data (i.e. edge ratio, infection probability and so on) we want to predict the number of zombies and survivors. In other words: we have a regression task.

Let's start by loading the data:

In [5]:
import numpy as np
import keras
import pandas as pd
import os

In [6]:
# Load the data
data_fname = os.path.join('shared', 'za_processed.h5')
with pd.HDFStore(data_fname) as store:
    data_tr = store['data_tr']
    data_vl = store['data_vl']
    data_ts = store['data_ts']
    means_in = store['means_in']
    stds_in = store['stds_in']
    sim_in = store['sim_in']
    sim_out = store['sim_out']
    in_defaults = store['in_defaults']
    pop_size = store['meta']['pop_size']

data_tr.head()

,edge_ratio,inf_prob,act_rate,rec_rate,ds_rate,di_rate,dr_rate,time,s_num,i_num,r_num,num,survivors
36246,-0.710919,-1.219620,-0.001475,0.505277,1.226580,-1.228638,0.05,7,369,0.008,0,373,0.738
12266,-0.004578,-1.219620,-1.226457,-0.168870,-1.224009,1.220550,0.00,7,480,0.028,1,495,0.962
45896,1.408103,-1.219620,-0.001475,-0.168870,1.226580,1.220550,0.05,7,124,0.364,10,316,0.268
40931,-0.004578,0.003019,1.223508,0.505277,1.226580,-0.004044,0.05,7,67,0.500,36,353,0.206
47950,1.408103,1.225658,-0.001475,-1.517164,1.226580,-0.004044,0.05,7,0,0.624,0,312,0.000


## Separating Input and Output

Since we stored the series of input and output attributes, it is fairly easy to separate out data and have it ready for training:

In [7]:
# Separate input and output
X_tr = data_tr[sim_in]
Y_tr = data_tr[sim_out]
X_vl = data_vl[sim_in]
Y_vl = data_vl[sim_out]
X_ts = data_ts[sim_in]
Y_ts = data_ts[sim_out]

## Learning Networks

We are now ready to learn a ML model: we will start by training a network for each type of output (zombies and survivors), using the keras `Sequential` API.

Our loss function will be the Mean Squared Error (as it is often the case for regression problems) and we will also evaluate our networks using the R2 score (i.e. coefficient of determination).

We will use a simple callback to stop trainig if no improvement is reported on the validation set for a number of iterations, and a second callback to store the best model on a pair of files.

For more information about how to use all these components, see [the keras documentation](https://keras.io/getting-started/sequential-model-guide/)

Let's start with a very simple, linear model (if we are lucky it may be enough):

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
import tensorflow
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Seed the RNGs
np.random.seed(42)
tensorflow.set_random_seed(42)

# Custom R2 metric (courtesy of https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/34019)
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Input shape
input_shape = (X_tr.shape[1],)

# Handle the two outputs
max_epochs = 50
for target in sim_out:
    y_tr = Y_tr[target].values
    y_vl = Y_vl[target].values
    y_ts = Y_ts[target].values

    # Define a Neural Network model to predict the number of infected
    model = Sequential()
    model.add(Dense(1, activation='linear', input_shape=input_shape))

    # Compile
    model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=[r2_score])

    # Setup and perform training
    weight_fname = os.path.join('shared', 'nn_reg_%s.h5' % target)
    callbacks = [EarlyStopping(monitor='val_loss', patience=5),
                 ModelCheckpoint(filepath=weight_fname, monitor='val_loss', save_best_only=True)]
    model.fit(X_tr, y_tr, epochs=max_epochs, batch_size=32, callbacks=callbacks,
              validation_data=(X_vl, y_vl), verbose=0)
    
    # Save the model architecture
    arch_fname = os.path.join('shared', 'nn_reg_%s.json' % target)
    with open(arch_fname, 'w') as fp:
        fp.write(model.to_json())

    # Evaluate on the test set
    print('=== Results for target "{}"'.format(target))
    res_tr = model.evaluate(X_tr, y_tr, batch_size=len(X_tr))
    print('Loss and R2 on the training set: {}, {}'.format(*res_tr))
    res_vl = model.evaluate(X_vl, y_vl, batch_size=len(X_vl))
    print('Loss and R2 on the validation set: {}, {}'.format(*res_vl) )
    res_ts = model.evaluate(X_ts, y_ts, batch_size=len(X_ts))
    print('Loss and R2 on the test set: {}, {}'.format(*res_ts))

=== Results for target "i_num"
32400/32400 [==============================] - 0s 0us/step
Loss and R2 on the training set: 0.03768403083086014, 0.5470709800720215
8100/8100 [==============================] - 0s 1us/step
Loss and R2 on the validation set: 0.03885703161358833, 0.5265119075775146
8100/8100 [==============================] - 0s 1us/step
Loss and R2 on the test set: 0.0392201766371727, 0.5309156179428101
=== Results for target "survivors"
32400/32400 [==============================] - 0s 0us/step
Loss and R2 on the training set: 0.04821530357003212, 0.5605478286743164
8100/8100 [==============================] - 0s 0us/step
Loss and R2 on the validation set: 0.049723319709300995, 0.5361647009849548
8100/8100 [==============================] - 0s 0us/step
Loss and R2 on the test set: 0.05035172030329704, 0.5415189266204834


Not particualarly good... Let's stack more layers!

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
import tensorflow
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Seed the RNGs
np.random.seed(42)
tensorflow.set_random_seed(42)

# Custom R2 metric (courtesy of https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/34019)
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Input shape
input_shape = (X_tr.shape[1],)

# Handle the two outputs
max_epochs = 50
for target in sim_out:
    y_tr = Y_tr[target].values
    y_vl = Y_vl[target].values
    y_ts = Y_ts[target].values

    # Define a Neural Network model to predict the number of infected
    model = Sequential()
    model.add(Dense(16, activation='relu', input_shape=input_shape))
    model.add(Dense(1, activation='linear'))

    # Compile
    model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=[r2_score])

    # Setup and perform training
    weight_fname = os.path.join('shared', 'nn_reg_%s.h5' % target)
    callbacks = [EarlyStopping(monitor='val_loss', patience=5),
                 ModelCheckpoint(filepath=weight_fname, monitor='val_loss', save_best_only=True)]
    model.fit(X_tr, y_tr, epochs=max_epochs, batch_size=32, callbacks=callbacks,
              validation_data=(X_vl, y_vl), verbose=0)
    
    # Save the model architecture
    arch_fname = os.path.join('shared', 'nn_reg_%s.json' % target)
    with open(arch_fname, 'w') as fp:
        fp.write(model.to_json())

    # Evaluate on the test set
    print('=== Results for target "{}"'.format(target))
    res_tr = model.evaluate(X_tr, y_tr, batch_size=len(X_tr))
    print('Loss and R2 on the training set: {}, {}'.format(*res_tr))
    res_vl = model.evaluate(X_vl, y_vl, batch_size=len(X_vl))
    print('Loss and R2 on the validation set: {}, {}'.format(*res_vl) )
    res_ts = model.evaluate(X_ts, y_ts, batch_size=len(X_ts))
    print('Loss and R2 on the test set: {}, {}'.format(*res_ts))

=== Results for target "i_num"
32400/32400 [==============================] - 0s 0us/step
Loss and R2 on the training set: 0.028682636097073555, 0.6552598476409912
8100/8100 [==============================] - 0s 0us/step
Loss and R2 on the validation set: 0.030186014249920845, 0.632171630859375
8100/8100 [==============================] - 0s 0us/step
Loss and R2 on the test set: 0.0308137945830822, 0.6314582824707031


Even more...

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
import tensorflow
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Seed the RNGs
np.random.seed(42)
tensorflow.set_random_seed(42)

# Custom R2 metric (courtesy of https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/34019)
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Input shape
input_shape = (X_tr.shape[1],)

# Handle outputs
max_epochs = 50
for target in sim_out:
    y_tr = Y_tr[target].values
    y_vl = Y_vl[target].values
    y_ts = Y_ts[target].values

    # Define a Neural Network model to predict the number of infected
    model = Sequential()
    model.add(Dense(16, activation='relu', input_shape=input_shape))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))

    # Compile
    model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=[r2_score])

    # Setup and perform training
    weight_fname = os.path.join('shared', 'nn_reg_%s.h5' % target)
    callbacks = [EarlyStopping(monitor='val_loss', patience=5),
                 ModelCheckpoint(filepath=weight_fname, monitor='val_loss', save_best_only=True)]
    model.fit(X_tr, y_tr, epochs=max_epochs, batch_size=32, callbacks=callbacks,
              validation_data=(X_vl, y_vl), verbose=0)
    
    # Save the model architecture
    arch_fname = os.path.join('shared', 'nn_reg_%s.json' % target)
    with open(arch_fname, 'w') as fp:
        fp.write(model.to_json())

    # Evaluate on the test set
    print('=== Results for target "{}"'.format(target))
    res_tr = model.evaluate(X_tr, y_tr, batch_size=len(X_tr))
    print('Loss and R2 on the training set: {}, {}'.format(*res_tr))
    res_vl = model.evaluate(X_vl, y_vl, batch_size=len(X_vl))
    print('Loss and R2 on the validation set: {}, {}'.format(*res_vl) )
    res_ts = model.evaluate(X_ts, y_ts, batch_size=len(X_ts))
    print('Loss and R2 on the test set: {}, {}'.format(*res_ts))

=== Results for target "i_num"
32400/32400 [==============================] - 0s 0us/step
Loss and R2 on the training set: 0.028382619842886925, 0.6588658094406128
8100/8100 [==============================] - 0s 0us/step
Loss and R2 on the validation set: 0.02973398193717003, 0.6376798152923584
8100/8100 [==============================] - 0s 1us/step
Loss and R2 on the test set: 0.03037528693675995, 0.6367030143737793
=== Results for target "survivors"
32400/32400 [==============================] - 0s 0us/step
Loss and R2 on the training set: 0.038163572549819946, 0.652163028717041
8100/8100 [==============================] - 0s 0us/step
Loss and R2 on the validation set: 0.040193766355514526, 0.6250594854354858
8100/8100 [==============================] - 0s 1us/step
Loss and R2 on the test set: 0.04068506881594658, 0.6295392513275146


It seems we have reached a plateau, but we are still not doing well.

What happened?

Well, several things. Let's start with one of them...

## About Correlations

The two outputs that we care about (zombies and survivors) are quite obviously correlated. In case of no deaths, their sum is always necessarily equal to the total population, i.e. 500 individuals (or 1, after the normalization).

By learning individual models for the two outputs, we are failing to provide the network with this information.

We can try and address this issue by predicting **both outputs at the same time**:

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
import tensorflow
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Seed the RNGs
np.random.seed(42)
tensorflow.set_random_seed(42)

# Custom R2 metric (courtesy of https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/34019)
def r2_score(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Input shape
input_shape = (X_tr.shape[1],)

# Both outputs at the same time
max_epochs = 50

# Define a Neural Network model to predict the number of infected
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=input_shape))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='linear'))

# Compile
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=[r2_score])

# Setup and perform training
weight_fname = os.path.join('shared', 'nn_reg.h5')
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath=weight_fname, monitor='val_loss', save_best_only=True)]
model.fit(X_tr, Y_tr, epochs=max_epochs, batch_size=32, callbacks=callbacks,
          validation_data=(X_vl, Y_vl), verbose=0)

# Save the model architecture
arch_fname = os.path.join('shared', 'nn_reg.json')
with open(arch_fname, 'w') as fp:
    fp.write(model.to_json())

# Evaluate on the test set
res_tr = model.evaluate(X_tr, Y_tr, batch_size=len(X_tr))
print('Loss and R2 on the training set: {}, {}'.format(*res_tr))
res_vl = model.evaluate(X_vl, Y_vl, batch_size=len(X_vl))
print('Loss and R2 on the validation set: {}, {}'.format(*res_vl) )
res_ts = model.evaluate(X_ts, Y_ts, batch_size=len(X_ts))
print('Loss and R2 on the test set: {}, {}'.format(*res_ts))

32400/32400 [==============================] - 0s 0us/step
Loss and R2 on the training set: 0.03323161602020264, 0.723670244216919
8100/8100 [==============================] - 0s 1us/step
Loss and R2 on the validation set: 0.03502108156681061, 0.7078489065170288
8100/8100 [==============================] - 0s 1us/step
Loss and R2 on the test set: 0.03566659986972809, 0.7082128524780273


Considerably better! But still not particularly good.

What is going on? Let's go back to the drawing board, i.e. to the data pre-processing step.